# Installation

In [ ]:
# !pip install pythreejs
# !pip install numpy, scipy
# !pip install notebook==6.5.4
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user

In [ ]:

import numpy as np
import time

# Particle

In [ ]:
class Particle:
    def __init__(self, pos, vel, mass):
        self.position = pos
        self.velocity = vel
        self.force = np.array([0, 0, 0])
        self.mass = mass
        
    def clear_force(self):
        self.force = np.array([0, 0, 0])

# Forces

In [ ]:
# Types of forces
# 1. Constant e.g. gravity
# 2. position dependent e.g. forces fields, winds
# 3. velocity dependent e.g. drag, friction
# 4. n-ary e.g. springs
# 5. collision

class Force:
    def apply(self, particles):
        pass

class Gravity(Force):
    def __init__(self, gravity=np.array([0, -9.8, 0])):
        self.G = gravity
        
    def apply(self, particles):
        for p in particles:
            p.force += p.mass * self.G

class Drag(Force):
    def __init__(self, k_drag=0.1):
        self.k_drag = k_drag
        
    def apply(self, particles):
        for p in particles:
            p.force += -self.k_drag * p.velocity

class Spring(Force):
    def __init__(self, particle1, particle2,  k_s, k_d, l0):
        self.p1 = particle1
        self.p2 = particle2
        self.k_s = k_s
        self.k_d = k_d
        self.l0 = l0
        
    def apply(self, particles):
        x1 = self.p1.position
        x2 = self.p2.position
        
        l = x1 - x2
        l_dot = self.p1.velocity - self.p2.velocity
        length = np.linalg.norm(l)
        
        f = -(self.k_s * (length - self.l0) + self.k_d * np.dot(l_dot, l) / length) * l / length
        
        self.p1.force += f
        self.p2.force += -f

# Particle System

In [ ]:
class ParticleSystem:
    def __init__(self, particles=None, forces=None,):
        
        self.particles = []
        self.forces = []
        
        
    def add_particle(self, particle):
        if isinstance(particle, list):
            self.particles.extend(particle)
            
        elif isinstance(particle, Particle):       
            self.particles.append(particle)
        
        else:
            raise ValueError("Invalid particle type")
        
    def add_force(self, force):
        if isinstance(force, list):
            self.forces.extend(force)
            
        elif isinstance(force, Force):       
            self.forces.append(force)
        
        else:
            raise ValueError("Invalid force type")
    
    def evaluate_derivative(self,):
        # (1) Loop over particles, zero force accumulators
        for p in self.particles:
            p.clear_force()
        
        # (2) Calculate forces by invoking apply functions, sum all forces into accumulators
        for f in self.forces:
            f.apply(self.particles)
        
    
    def render(self,):
        pass

# Numerical Method

In [ ]:
class Integrator:
    def solve(self, particle_system, time_step):
        pass
    
class Euler(Integrator):
    def solve(self, particle_system, time_step):
        
        particle_system.evaluate_derivative()
        
        for p in particle_system.particles:
            a = p.force / p.mass
            p.velocity += a * time_step
            p.position += p.velocity * time_step
            
    def solve_adaptive(self, particle_system, init_time_step, tol):
        #TODO
        pass
            

class Midpoint(Integrator):
    def solve(self, particle_system, time_step):
        
        # Save initial position and velocity
        init_position = [p.position.copy() for p in particle_system.particles]
        init_velocity = [p.velocity.copy() for p in particle_system.particles]        
        
        particle_system.evaluate_derivative()
        
        # Compute midpoint position and velocity
        for p in particle_system.particles:
            a = p.force / p.mass
            p.velocity = p.velocity + a * time_step / 2
            p.position = p.position + p.velocity * time_step / 2
        
        # Compute forces at midpoint
        particle_system.evaluate_derivative()
        
        # Compute final position and velocity
        for i, p in enumerate(particle_system.particles):
            a = p.force / p.mass
            p.velocity = init_velocity[i] + a * time_step
            p.position = init_position[i] + p.velocity * time_step
        
class RK4(Integrator):
    def solve(self, particle_system, time_step):
        pass


# Render

In [ ]:
from pythreejs import * 
from IPython.display import display
import ipywidgets as widgetsf
from ipyevents import Event

camera = OrthographicCamera(
    left=-10, right=10, top=10, bottom=-10, near=0.1, far=100
)
camera.position = [0, 0, 10]  # Place camera in front of the scene.
camera.lookAt([0, 0, 0])

# Create a scene
scene = Scene()


# ----------------------------
# CREATE THE PARTICLE SYSTEM
# ----------------------------

# For our 2D particle system we generate a set of particles in the XY plane (with z=0)
num_particles = 50
positions = np.random.uniform(-5, 5, (num_particles, 3))
positions[:, 2] = 0  # Ensure particles lie in the XY plane

# Create a BufferGeometry with these positions.
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(positions, normalized=False)
    }
)

# Create a PointsMaterial to render the particles (red circles)
material = PointsMaterial(color='red', size=10.0)

# Create the Points object to add to the scene.
points = Points(geometry=geometry, material=material)
scene.add(points)


# ----------------------------
# SET UP THE RENDERER
# ----------------------------

renderer = Renderer(
    scene=scene, 
    camera=camera, 
    controls=[OrbitControls(controlling=camera, enableRotate=False)],
    width=600, height=600
)


# ----------------------------
# MOUSE PICKER / DRAG HANDLING WITH IPYEVENTS
# ----------------------------

# Global variable to store the index of the particle being dragged (if any)
dragging_particle_index = None

def screen_to_world(x, y, renderer=renderer, camera=camera):
    """
    Convert screen coordinates (from the renderer) to world coordinates.
    For an orthographic camera, this conversion is linear.
    """
    width = renderer.width
    height = renderer.height
    # Convert x, y to normalized device coordinates (NDC) in [-1, 1]
    ndc_x = (x / width) * 2 - 1
    ndc_y = -((y / height) * 2 - 1)  # flip y axis (top=0 vs. bottom=0)
    
    # For an orthographic camera, the world coordinate is given by a linear interpolation:
    world_x = ndc_x * (camera.right - camera.left) / 2 + (camera.left + camera.right) / 2
    world_y = ndc_y * (camera.top - camera.bottom) / 2 + (camera.top + camera.bottom) / 2
    return world_x, world_y

def on_mouse_event(change):
    """
    Handle mousedown, mousemove, and mouseup events from the renderer.
    """
    global dragging_particle_index
    event_type = change['type']
    x = change['relativeX']
    y = change['relativeY']
    
    if event_type == 'mousedown':
        # Convert click position to world coordinates
        world_x, world_y = screen_to_world(x, y)
        # Access particle positions from geometry attribute
        pos_array = points.geometry.attributes['position'].array
        # Compute distance (in the XY plane) from the click to each particle
        distances = np.sqrt((pos_array[:, 0] - world_x)**2 + (pos_array[:, 1] - world_y)**2)
        # Find the particle with the smallest distance
        min_index = np.argmin(distances)
        # Set a threshold for picking; if the click is close enough to a particle, pick it.
        if distances[min_index] < 0.5:
            dragging_particle_index = min_index
    
    elif event_type == 'mousemove':
        if dragging_particle_index is not None:
            world_x, world_y = screen_to_world(x, y)
            pos_array = points.geometry.attributes['position'].array
            # Update the dragged particle's position; we leave z unchanged.
            pos_array[dragging_particle_index, 0] = world_x
            pos_array[dragging_particle_index, 1] = world_y
            # Update the geometry and flag it for update.
            points.geometry.attributes['position'].array = pos_array
            points.geometry.attributes['position'].needsUpdate = True
            
    elif event_type == 'mouseup':
        dragging_particle_index = None

# Create an ipyevents Event on the renderer's DOM element to capture mouse events.
mouse_event = Event(
    source=renderer, 
    watched_events=['mousedown', 'mousemove', 'mouseup']
)
mouse_event.observe(on_mouse_event, names='data')

In [ ]:
display(renderer)

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Output, VBox, jslink

view_width = 600
view_height = 400

def find_minima(f, start=(0, 0), xlim=None, ylim=None):
    rate = 0.1 # Learning rate
    max_iters = 200 # maximum number of iterations
    iters = 0 # iteration counter
    
    cur = np.array(start[:2])
    previous_step_size = 1 #
    cur_val = f(cur[0], cur[1]) 
    
    while (iters < max_iters and
           xlim[0] <= cur[0] <= xlim[1] and ylim[0] <= cur[1] <= ylim[1]):
        iters = iters + 1
        candidate = cur - rate * (np.random.rand(2) - 0.5)
        candidate_val = f(candidate[0], candidate[1])
        if candidate_val >= cur_val:
            continue   # Bad guess, try again
        prev = cur
        cur = candidate
        cur_val = candidate_val
        previous_step_size = np.abs(cur - prev)
        yield tuple(cur) + (cur_val,)

    print("The local minimum occurs at", cur)
    
def f(x, y):
    return x ** 2 + y ** 2


nx, ny = (20, 20)  # grid resolution
xmax = 1           # grid extent (+/-)
x = np.linspace(-xmax, xmax, nx)
y = np.linspace(-xmax, xmax, ny)
step = x[1] - x[0]
xx, yy = np.meshgrid(x, y)
# Grid lattice values:
grid_z = np.vectorize(f)(xx, yy)
# Grid square center values:
center_z = np.vectorize(f)(0.5 * step + xx[:-1,:-1], 0.5 * step + yy[:-1,:-1])

# Surface geometry:
surf_g = SurfaceGeometry(z=list(grid_z.flat), 
                         width=2 * xmax,
                         height=2 * xmax,
                         width_segments=nx - 1,
                         height_segments=ny - 1)

# Surface material. Note that the map uses the center-evaluated function-values:
surf = Mesh(geometry=surf_g,
            material=MeshLambertMaterial(map=height_texture(center_z, 'YlGnBu_r')))

# Grid-lines for the surface:
surfgrid = SurfaceGrid(geometry=surf_g, material=LineBasicMaterial(color='black'),
                       position=[0, 0, 1e-2])  # Avoid overlap by lifting grid slightly



# Set up scene:
key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.4)
c = PerspectiveCamera(position=[0, 3, 3], up=[0, 0, 1], aspect=view_width / view_height,
                      children=[key_light])

scene = Scene(children=[surf, c, surfgrid, AmbientLight(intensity=0.8)])

renderer = Renderer(camera=c, scene=scene,
                    width=view_width, height=view_height,
                    controls=[OrbitControls(controlling=c)])

out = Output()        # An Output for displaying captured print statements
box = VBox([renderer])
display(box)

# Picker object
hover_picker = Picker(controlling=surf, event='mousemove')
renderer.controls = renderer.controls + [hover_picker]

# A sphere for representing the current point on the surface
hover_point = Mesh(geometry=SphereGeometry(radius=0.05),
                   material=MeshLambertMaterial(color='hotpink'))
scene.add(hover_point)

# Have sphere follow picker point:
jslink((hover_point, 'position'), (hover_picker, 'point'));

coord_label = HTML()  # A label for showing hover picker coordinates

def on_hover_change(change):
    coord_label.value = 'Pink point at (%.3f, %.3f, %.3f)' % tuple(change['new'])

on_hover_change({'new': hover_point.position})
hover_picker.observe(on_hover_change, names=['point'])
box.children = (coord_label,) + box.children

# Create our picker for the double-click event ("dblclick")
click_picker = Picker(controlling=surf, event='dblclick')

def on_click(change):
    value = change['new']
    with out:
        print('Clicked on %s' % (value,))

    # Add a red sphere on the picked point
    point = Mesh(geometry=SphereGeometry(radius=0.05), 
                 material=MeshLambertMaterial(color='red'),
                 position=value)
    scene.add(point)
    
    # Plot solution as a red line, this will start out empty
    points = [value]
    line = Line2(geometry=LineGeometry(positions=points), material=LineMaterial(color='red', linewidth=2))
    scene.add(line)
    with out:  # Pick up any print statements in the algorithm
        for pt in find_minima(f, value, [-xmax, xmax], [-xmax, xmax]):
            # For each point, update the line:
            pt = list(pt)
            pt[2] += 1e-2   # offset to clear surface
            line.geometry = LineGeometry(positions=np.vstack([line.geometry.positions, pt]))


# When the point selected by the picker changes, trigger our function:
click_picker.observe(on_click, names=['point'])

# Update figure:
renderer.controls = renderer.controls + [click_picker]
box.children = box.children + (out,)

In [5]:
from pythreejs import *
import ipywidgets as widget
from IPython.display import display

perspective = False
container = Mesh(geometry=Geometry(), 
               material=MeshBasicMaterial(),
               position=[0, 0, 0])

ball1 = Mesh(geometry=SphereGeometry(radius=1), 
                                    material=MeshLambertMaterial(color='red'),
                                    position=[-.5, 0, 1])
ball2 = Mesh(geometry=SphereGeometry(radius=1), 
                                    material=MeshLambertMaterial(color='blue'),
                                    position=[.5, 0, 0])
container.children=[ball1,ball2]

scene = Scene(children=[container, AmbientLight(color='#777777')])

if perspective:
    c = PerspectiveCamera(position=[0, 0, 10], up=[0, 0, 1],aspect=1.0)
else:
    c = OrthographicCamera(position=[0, 0, 10], up=[0, 0, 1],
                          left=-2,right=2,top=2,bottom=-2,near=.1,far=1000)
    
c.children=[DirectionalLight(color='white',
                position=[3, 5, 1],intensity=0.5)]
renderer = Renderer(camera=c, width='300',height='300',
                    scene=scene, 
                    controls=[OrbitControls(controlling=c)])

click_picker = Picker(root=scene.children[0], event='mousemove')
infobox = widget.HTML()
def change_info(change):
    infobox.value = 'Color: {}'.format(click_picker.object.material.color)
click_picker.observe(change_info, names=['object'])

renderer.controls = renderer.controls + [click_picker]

display(infobox)
display(renderer)

TraitError: The 'controlling' trait of a Picker instance expected an Object3D, not the NoneType None.